In [ ]:
import sys
import os
import numpy as np
import torch
from PIL import features  
import pydicom
import cv2
import matplotlib.pyplot as plt
import logging
from scipy.ndimage import zoom
import traceback
import csv

In [ ]:
def visualize_mask_slice(ct_array, mask_array, slice_idx=None, roi_name="", save_path=None):
    """Visualize a mask overlay on CT slice"""
    if slice_idx is None:
        # Find slice with most mask content
        mask_sums = np.sum(mask_array, axis=(1, 2))
        if np.max(mask_sums) > 0:
            slice_idx = np.argmax(mask_sums)
        else:
            slice_idx = mask_array.shape[0] // 2
    
    plt.figure(figsize=(8, 8))
    plt.imshow(ct_array[slice_idx], cmap='gray')
    plt.imshow(mask_array[slice_idx], cmap='hot', alpha=0.5)
    plt.title(f'Mask {roi_name} (Slice {slice_idx}) 🎭')
    plt.axis('off')
    
    if save_path:
        plt.savefig(save_path, dpi=150)
        plt.close()
    else:
        plt.show()

def visualize_dose_slice(ct_array, dose_array, slice_idx=None, save_path=None):
    """Visualize a dose overlay on CT slice"""
    if slice_idx is None:
        # Find slice with most dose content
        dose_sums = np.sum(dose_array, axis=(1, 2))
        if np.max(dose_sums) > 0:
            slice_idx = np.argmax(dose_sums)
        else:
            slice_idx = dose_array.shape[0] // 2
    
    plt.figure(figsize=(10, 5))
    
    # Show CT
    plt.subplot(1, 2, 1)
    plt.imshow(ct_array[slice_idx], cmap='gray')
    plt.title(f'CT (Slice {slice_idx}) 🖼️')
    plt.axis('off')
    
    # Show Dose overlay on CT
    plt.subplot(1, 2, 2)
    plt.imshow(ct_array[slice_idx], cmap='gray')
    plt.imshow(dose_array[slice_idx], cmap='jet', alpha=0.7)
    plt.title(f'Dose Overlay (Slice {slice_idx}) 💊')
    plt.colorbar(label='Dose (Gy)')
    plt.axis('off')
    
    if save_path:
        plt.savefig(save_path, dpi=150)
        plt.close()
    else:
        plt.show()

def combine_masks(output_path, target_rois):
    """Combine all masks"""
    for patient_id in os.listdir(output_path):
        patient_path = os.path.join(output_path, patient_id)
        if not os.path.isdir(patient_path):
            continue

        print(f"🔄 Combining masks for Patient: {patient_id}")

        combined_mask = None
        
        # Collect all masks
        for roi_name in target_rois:
            mask_path = os.path.join(patient_path, f"Mask_{roi_name}.npy")
            if not os.path.exists(mask_path):
                continue
                
            mask_array = np.load(mask_path)
            
            if combined_mask is None:
                combined_mask = mask_array.copy()
            else:
                combined_mask += mask_array

        if combined_mask is not None:
            # Ensure values are 0 or 1
            combined_mask = np.clip(combined_mask, 0, 1)
            # Save combined mask
            combined_mask_path = os.path.join(patient_path, "Combined_applicator.npy")
            np.save(combined_mask_path, combined_mask)
            
            # Visualize combined mask
            ct_path = os.path.join(patient_path, "CT_raw.npy")
            if os.path.exists(ct_path):
                ct_array = np.load(ct_path)
                vis_path = os.path.join(patient_path, "visualizations", "combined_applicator.png")
                visualize_mask_slice(ct_array, combined_mask, roi_name="Combined", save_path=vis_path)
                
            print(f"✅ Combined mask saved for Patient: {patient_id}")
        else:
            print(f"⚠️ No masks to combine for Patient: {patient_id}")

In [ ]:
output_directory = r"d:\ThesisMayn\Data_numpy\v2\Predict"  
    
target_rois = ['Applicator1','applicator1', 'Applicator2','applicator2','Applicator3','applicator3','Applicator4','applicator4',
                   'Applicator5','applicator5','Applicator6','applicator6','Applicator7','applicator7',
                   'Applicator8','applicator8','Applicator9','applicator9','Applicator10','applicator10',
                   'needdle1','needdle2','needdle3','needdle4','needdle5','needdle6']  